In [1]:
#Imports
import os
import sys
import glob
import torch

import torchvision
from tqdm import trange

import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import datetime as dt
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot   as plt
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import time
from datetime import datetime

from PIL import Image
from torch.utils.data  import Dataset
from torch.autograd    import Variable
from torch.optim       import lr_scheduler

from torch.utils.data  import Dataset, DataLoader, Subset
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision       import transforms, datasets, models
from os                import listdir, makedirs, getcwd, remove
from os.path           import isfile, join, abspath, exists, isdir, expanduser
from efficientnet_pytorch import EfficientNet


%matplotlib inline

In [2]:
origin_path = "./dataset"
train_path = join(origin_path, "train/train")
test_path = join(origin_path,"test/test")
extraimage_path = join(origin_path, "extraimages/extraimages")

In [3]:
# Transformations for both the training and testing data
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

# Do data transforms here, Try many others
train_transforms = A.Compose(
        [
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            A.Rotate(),
            A.Resize(height=256, width=256, p=1),
            A.Cutout(num_holes=8, max_h_size=32, max_w_size=32, fill_value=0, p=0.5),
             A.ToFloat(max_value=255.0),
            ToTensorV2(p=1.0),
        ]
    )

test_transforms = A.Compose(
        [
            A.Resize(height=256, width=256, p=1),
             A.ToFloat(max_value=255.0),
            ToTensorV2(p=1.0),
        ]
    )

/opt/anaconda3/lib/python3.11/site-packages/albumentations/augmentations/dropout/cutout.py:49: FutureWarning: Cutout has been deprecated. Please use CoarseDropout
  warnings.warn(


In [4]:
class CassavaDataset(Dataset):
    def __init__(self, path, transform=None):
        self.classes = os.listdir(path)
        self.path = [f"{path}/{className}" for className in self.classes]
        self.file_list = [glob.glob(f"{x}/*") for x in self.path]
        self.transform = transform
        self.classes_dict = {}

        files = []
        for i, className in enumerate(self.classes):
            for fileName in self.file_list[i]:
                self.classes_dict[i] = className
                files.append([i, className, fileName])
        self.file_list = files
        files = None

    def __len__(self):
        return len(self.file_list)
    
    def get_image_filename(self, idx):
        file_path = self.file_list[idx][2]
        file_name = file_path.split("/")[-1]
        return file_name

    def __getitem__(self, idx):
        fileName = self.file_list[idx][2]
        classCategory = self.file_list[idx][0]
        im = Image.open(fileName)
        if self.transform:
            im = self.transform(image= np.asarray(im))["image"]
            
        return im, classCategory

In [5]:
train_data = CassavaDataset(train_path, transform=train_transforms)
test_data = CassavaDataset(test_path, transform=test_transforms)

In [6]:
# Define the K-fold Cross Validator
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
BATCH_SIZE = 16
# K-fold Cross Validation model evaluation
data_loaders = []
for fold, (train_ids, val_ids) in enumerate(kfold.split(train_data)):
    train_dataset = Subset(train_data, train_ids)
    val_dataset = Subset(train_data, val_ids)

    # # Creating PT data samplers and loaders:
    # train_sampler = SubsetRandomSampler(train_ids)
    # valid_sampler = SubsetRandomSampler(val_ids)

    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True,
                                                 drop_last=True)
    valid_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False,
                                                 drop_last=True)

    data_loaders.append((train_loader, valid_loader))

In [7]:
validation_split = .2
shuffle_dataset = True
random_seed= 42

# Creating data indices for training and validation splits:
dataset_size = len(train_data)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))

if shuffle_dataset :
    np.random.seed(random_seed)
    np.random.shuffle(indices)

train_indices, val_indices = indices[split:], indices[:split]

In [8]:
# Device configuration
device = torch.device('mps' if torch.backends.mps.is_available() else 'cpu')
device

device(type='mps')

In [9]:
def make_model():
    model = EfficientNet.from_pretrained('efficientnet-b4') 
    model._fc = nn.Linear(in_features=1792, out_features=5, bias=True)
    return model

model = make_model()

Loaded pretrained weights for efficientnet-b4


In [10]:
# Define Models 

class Classifier(nn.Module):
    def __init__(self, num_classes):
        super(Classifier, self).__init__()
        # Block 1
        self.conv1 = nn.Conv2d(in_channels = 3, out_channels = 32, kernel_size = 3, stride = 2, padding = 1)
        self.relu1 = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size = 2)

        #Block 2
        self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, stride = 2)
        self.relu2 = nn.ReLU()
        self.maxpool2 = nn.MaxPool2d(kernel_size=2)

        #Block 3
        self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 32, kernel_size = 3, stride = 2)
        self.relu3 = nn.ReLU()
        self.maxpool3 = nn.MaxPool2d(kernel_size=2)

        # last fully-connected layer
        self.fc = nn.Linear(32*3*3, num_classes)


    def forward(self, input):

        x = self.maxpool1(self.relu1(self.conv1(input)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = self.maxpool3(self.relu3(self.conv3(x)))

        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


# model = Classifier(5)

In [12]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = lr_scheduler.OneCycleLR(
                                    optimizer,
                                    max_lr=0.001,
                                    epochs=30,
                                    steps_per_epoch=int(len(train_data) / BATCH_SIZE), 
                                    pct_start=0.1,
                                    anneal_strategy='cos',
                                    final_div_factor=10**5)

In [13]:
def train(model, data_loader, optimizer, scheduler, num_epochs):
    """Simple training loop for a PyTorch model.""" 
    
    # Make sure model is in training mode.
    model.train()
    
    # Move model to the device (CPU or MPS or GPU).
    model.to(device)
    
    # Exponential moving average of the loss.
    ema_loss = 0

    print('----- Training Loop -----')
    # Loop over epochs.
    for epoch in trange(num_epochs):
      for fold, (train_loader,valid_loader) in enumerate(data_loader):
        correct = 0  # Reset correct for each fold
        # Loop over data.
        for _, (features, target) in enumerate(train_loader):
              
          # Forward pass.
          output = model(features.to(device))
          loss = criterion(output.to(device), target.to(device))


          # Backward pass.
          optimizer.zero_grad()
          loss.backward()
          optimizer.step()
          scheduler.step()

        # Evaluate on the validation set
        model.eval()
        total_valid_loss = 0
        with torch.no_grad():
            for batch_idx, (features, target) in enumerate(valid_loader):
                output = model(features.to(device))
                loss = criterion(output.to(device), target.to(device))
                # Get the label corresponding to the highest predicted probability.
                pred = output.argmax(dim=1, keepdim=True)
                # Count number of correct predictions.
                correct += pred.cpu().eq(target.view_as(pred)).sum().item()
                total_valid_loss += loss.item()
        valid_loss_avg = total_valid_loss / len(valid_loader)
        # NOTE: It is important to call .item() on the loss before summing.
        if ema_loss is None:
            ema_loss = loss.item()
        else:
            ema_loss += (loss.item() - ema_loss) * 0.01
      
        print(f'Epoch: {epoch}, Fold: {fold}, Train Loss: {ema_loss:.6f}, Valid Loss: {valid_loss_avg:.6f}')
        # Print test accuracy.
        percent = 100. * correct / len(valid_loader.dataset)  # Use valid_loader.dataset instead of data_loader.dataset
        print(f'Validation accuracy: {correct} / {len(valid_loader.dataset)} ({percent:.0f}%)')
        torch.save(model.state_dict(), 'model.ckpt')
        # Reset model to training mode
        model.train()

In [14]:
def test(model, data_loader):
    """Measures the accuracy of a model on a data set.""" 
    # Make sure the model is in evaluation mode.
    model.eval()
    correct = 0
    print('----- Model Evaluation -----')
    # We do not need to maintain intermediate activations while testing.
    with torch.no_grad():
        
        # Loop over test data.
        for images, target in data_loader:
            # Forward pass.
            output = model(images.to(device))
            
            # Get the label corresponding to the highest predicted probability.
            pred = output.argmax(dim=1, keepdim=True)
            # Count number of correct predictions.
            correct += pred.cpu().eq(target.view_as(pred)).sum().item()

    # Print test accuracy.
    percent = 100. * correct / len(data_loader.dataset)
    print(f'Validation accuracy: {correct} / {len(data_loader.dataset)} ({percent:.0f}%)')
    torch.save(model.state_dict(), 'model.ckpt')
    return percent

In [15]:
num_epochs = 5
train(model, data_loaders, optimizer, scheduler, num_epochs=num_epochs)

----- Training Loop -----


  0%|          | 0/5 [00:00<?, ?it/s]

Epoch: 0, Fold: 0, Train Loss: 0.006904, Valid Loss: 0.676838
Validation accuracy: 836 / 1132 (74%)
Epoch: 0, Fold: 1, Train Loss: 0.013317, Valid Loss: 0.637348
Validation accuracy: 871 / 1131 (77%)
Epoch: 0, Fold: 2, Train Loss: 0.016681, Valid Loss: 0.781420
Validation accuracy: 865 / 1131 (76%)
Epoch: 0, Fold: 3, Train Loss: 0.019841, Valid Loss: 0.597935
Validation accuracy: 884 / 1131 (78%)
Epoch: 0, Fold: 4, Train Loss: 0.029914, Valid Loss: 1.183779
Validation accuracy: 722 / 1131 (64%)


 20%|██        | 1/5 [17:37<1:10:28, 1057.03s/it]

Epoch: 1, Fold: 0, Train Loss: 0.043478, Valid Loss: 0.493725
Validation accuracy: 960 / 1132 (85%)
Epoch: 1, Fold: 1, Train Loss: 0.045935, Valid Loss: 0.345560
Validation accuracy: 992 / 1131 (88%)
Epoch: 1, Fold: 2, Train Loss: 0.047316, Valid Loss: 0.439324
Validation accuracy: 948 / 1131 (84%)
Epoch: 1, Fold: 3, Train Loss: 0.050744, Valid Loss: 0.389708
Validation accuracy: 978 / 1131 (86%)
Epoch: 1, Fold: 4, Train Loss: 0.057589, Valid Loss: 0.348040
Validation accuracy: 990 / 1131 (88%)


...

 60%|██████    | 3/5 [51:57<34:32, 1036.28s/it]

Epoch: 3, Fold: 0, Train Loss: 0.082688, Valid Loss: 0.225126
Validation accuracy: 1032 / 1132 (91%)
Epoch: 3, Fold: 1, Train Loss: 0.084420, Valid Loss: 0.201329
Validation accuracy: 1043 / 1131 (92%)
Epoch: 3, Fold: 2, Train Loss: 0.084486, Valid Loss: 0.322272
Validation accuracy: 1009 / 1131 (89%)
Epoch: 3, Fold: 3, Train Loss: 0.087249, Valid Loss: 0.303244
Validation accuracy: 996 / 1131 (88%)
Epoch: 3, Fold: 4, Train Loss: 0.087616, Valid Loss: 0.136828
Validation accuracy: 1067 / 1131 (94%)


 80%|████████  | 4/5 [3:19:56<1:08:52, 4132.25s/it]

Epoch: 4, Fold: 0, Train Loss: 0.087260, Valid Loss: 0.136417
Validation accuracy: 1067 / 1132 (94%)
Epoch: 4, Fold: 1, Train Loss: 0.087445, Valid Loss: 0.109469
Validation accuracy: 1076 / 1131 (95%)
Epoch: 4, Fold: 2, Train Loss: 0.090455, Valid Loss: 0.182552
Validation accuracy: 1051 / 1131 (93%)
Epoch: 4, Fold: 3, Train Loss: 0.091483, Valid Loss: 0.091418
Validation accuracy: 1088 / 1131 (96%)
Epoch: 4, Fold: 4, Train Loss: 0.091026, Valid Loss: 0.078240
Validation accuracy: 1092 / 1131 (97%)


100%|██████████| 5/5 [6:44:14<00:00, 4850.98s/it]  


In [16]:
test_loader = DataLoader(test_data, batch_size=BATCH_SIZE)

In [17]:
def predict(model, data_loader):
    model.eval()
    preds = []
    for images, _ in data_loader:
        # Forward pass.
        output = model(images.to(device))
        # Get the label corresponding to the highest predicted probability.
        pred = output.argmax(dim=1, keepdim=True)
        for p in pred:
            pred_class = train_data.classes_dict[p.item()]
            preds.append(pred_class)
    return preds

In [18]:
def generate_submission_file(predictions):
    mapped_preds = []
    for idx in range(len(test_data)):
        mapped_preds.append({'Category': predictions[idx], 'Id': test_data.get_image_filename(idx)})
    pd.DataFrame(mapped_preds).to_csv("sample_submission_file.csv", index=False)

In [19]:
predictions = predict(model, test_loader)
# Make submission here
generate_submission_file(predictions)